## Imports and Setup

In [1]:
import pandas as pd
import geopandas as gpd
import logging
import joblib
import torch

import os 
os.chdir('/home/itingzon.unicef/giga')
from src import sat_download
from utils import data_utils
from utils import config_utils
from utils import pred_utils

%load_ext autoreload
%autoreload 2

## Load Config Files

In [ ]:
iso_code="KEN"
adm_level="ADM2"
sum_threshold=5
buffer_size=150
spacing=150

config_file = os.path.join(os.getcwd(), "configs/data_configs/data_config_ISO_AF.yaml")
sat_config_file = os.path.join(os.getcwd(), "configs/sat_configs/sat_config_500x500_60cm.yaml")
sat_creds_file = os.path.join(os.getcwd(), "configs/sat_configs/issa_sat_creds.yaml")

config = config_utils.load_config(config_file)
sat_config = config_utils.load_config(sat_config_file)
sat_creds = config_utils.create_config(sat_creds_file)

## Generate Prediction Tiles

In [ ]:
tiles = pred_utils.batch_process_tiles(
    data_config, iso_code, spacing, buffer_size, sum_threshold, adm_level
)

## Download Satellite Images

In [ ]:
pred_utils.batch_download_sat_images(
    sat_config, sat_creds, config, iso_code, spacing, buffer_size, sum_threshold, adm_level
)

Gachoka 0/290 total tiles: (34337, 5)
Siakago 1/290 total tiles: (20121, 5)
Mwingi North 2/290 total tiles: (32835, 5)


100%|████████████████████| 32835/32835 [2:11:54<00:00,  4.15it/s]                                                       


Mwingi West 3/290 total tiles: (28527, 5)


100%|████████████████████| 28527/28527 [9:00:42<00:00,  1.14s/it]                                                       


Mwingi East 4/290 total tiles: (30893, 5)


 28%|█████▌              | 8655/30893 [3:27:22<6:41:26,  1.08s/it]                                                      

## Run Model

In [ ]:
model_config_file = os.path.join(cwd, f"configs/cnn_configs/vgg16.yaml")
model_config = config_utils.load_config(model_config_file)
result = pred_utils.cnn_predict(tiles, iso_code, shapename, model_config, sat_dir, n_classes=2)